# 1. predicting adsorption for each adsorption data point using Vt&BET only

In [1]:
import pandas as pd
import numpy as np
import os 
BET_path = 'C:/Kai_Zhang/MachineLearning/Unified gas Adsorption/Treated/E&E'

def pred_dataset(file_names):
    source_path = 'C:/Kai_Zhang/MachineLearning/Unified gas Adsorption/CO2_adsorption/new_data'
    train_df = pd.DataFrame()
    for file_name in file_names:
        temp_data = pd.read_excel(os.path.join(source_path,file_name+'-02-02-2022.xlsx'),skiprows= 1 )
        #temp_data = temp_data.dropna(axis=0,how = 'any',subset = feature_set)
        train_df = pd.concat([train_df,temp_data],axis=0)
       
    return train_df

In [33]:
from scipy import interpolate
import numpy as np
import pandas as pd

def inter_data(x_old,y_old):
    
    f = interpolate.interp1d(x_old, y_old,'slinear')    
    x_new = np.linspace(min(x_old), max(x_old),num=80)
       
    """
        for ele in x_old:
            t = bi_find(x_new,ele)

            x_new[t] = ele
        
        if (len(x_new)>150):
            x_new = x_new[len(x_new)-150:]
         = np.arange(min(x_old), max(x_old), (max(x_old)-min(x_old))/150)
    """
    y_new = f(x_new)
    #X_feature = np.append(x_new.reshape(-1,1),y_new.reshape(-1,1),axis=1).reshape(2,35).tolist()
    #x_mat = np.append(x_mat,X_feature,axis=1)
       
    return x_new, y_new

In [34]:
data = pred_dataset(["Ethane&Ethylene"])
data.head()

,Unnamed: 0,Literature,Literature_old,Unnamed: 1,GAC,GAS,E,S,A,B,...,%N,Temp(K),Pressure,Adsorp(mmol/g),Index,logP,logQ,logD,Label,Unnamed: 25
0,0,1,1.0,0,Maxsorb III,n-butane,0.0,0.0,0.0,0.0,...,NaN,328.0,0.2525,7.100826,2323,-1.376344,1.960211,3.336555,E&E,NaN
1,1,NaN,NaN,1,NaN,n-butane,0.0,0.0,0.0,0.0,...,NaN,328.0,0.3170,7.890571,2323,-1.148854,2.065669,3.214522,E&E,NaN
2,2,NaN,NaN,2,NaN,n-butane,0.0,0.0,0.0,0.0,...,NaN,328.0,0.3910,8.513421,2323,-0.939048,2.141644,3.080692,E&E,NaN
3,3,NaN,NaN,3,NaN,n-butane,0.0,0.0,0.0,0.0,...,NaN,328.0,0.4494,8.972815,2323,-0.799842,2.194199,2.994041,E&E,NaN
4,4,NaN,NaN,4,NaN,n-butane,0.0,0.0,0.0,0.0,...,NaN,328.0,0.6960,9.630076,2323,-0.362406,2.264891,2.627297,E&E,NaN


In [35]:

BET_df = pd.DataFrame()
cols = ["E","S","A","B","V","BET","Vt","Vmic","Vmeso","Temp(K)","Pressure","Adsorp(mmol/g)","Index"]
def bet_extraction(temp_df:pd.DataFrame):
    global BET_df
    literature = temp_df["Literature"].values[0]
    gac_name = temp_df["GAC"].values[0]
    print(literature,gac_name)
    
    try:
        if literature!=np.nan:
                bet_data = BET_path+'/'+str(literature)+"/"+gac_name+".csv"
                #print(bet_data)
                bet_curve = pd.read_csv(bet_data,header=None)
                
                x = bet_curve.iloc[:,0].values.tolist()
                y = bet_curve.iloc[:,1].values.tolist()
                _,new_y = inter_data(x,y)
                rows = len(temp_df)
                bet_matrix = [new_y for _ in range(rows)]
                bet_df = pd.DataFrame(bet_matrix,columns = ["BET_"+str(i) for i in range(len(new_y))])
                bet_df = pd.concat([bet_df,temp_df[cols].reset_index()],axis= 1,ignore_index=True)
                print(bet_df.shape)
                BET_df = pd.concat([BET_df,bet_df,],axis= 0,ignore_index=True)    
               

        else:
            print("There is not leterature for this gac: {}".format(gac_name))         
    except:
        print("The BET curve for gac: {} from paper {} is not available.".format(gac_name,literature))
total_index = list(set(data["Index"].values.tolist()))
for index in total_index:
    temp_df =  data[data["Index"]==index]
    bet_extraction(temp_df)

1 Maxsorb III
The BET curve for gac: Maxsorb III from paper 1 is not available.
nan nan
The BET curve for gac: nan from paper nan is not available.
nan nan
The BET curve for gac: nan from paper nan is not available.
nan nan
The BET curve for gac: nan from paper nan is not available.
EE2 PAC
(18, 94)
EE2 PAC
(14, 94)
EE2 PAC
(13, 94)
EE2 PAC
(7, 94)
nan nan
The BET curve for gac: nan from paper nan is not available.
nan nan
The BET curve for gac: nan from paper nan is not available.
EE4 C600-3
(12, 94)
EE4 C600-3
(12, 94)
EE4 C700-3
(12, 94)
EE4 C700-3
(12, 94)
EE4 C800-3
(12, 94)
EE4 C800-3
(12, 94)
EE4 C800-4
(12, 94)
EE4 C800-4
(12, 94)
EE4 C800-5
(12, 94)
EE4 C800-5
(11, 94)
EE4 C600-3
(11, 94)
EE4 C600-3
(11, 94)
EE4 C700-3
(10, 94)
EE4 C700-3
(11, 94)
EE4 C800-3
(12, 94)
EE4 C800-3
(12, 94)
EE4 C800-4
(9, 94)
EE4 C800-4
(11, 94)
EE4 C800-5
(10, 94)
EE4 C800-5
(10, 94)
EE4 C600-3
(9, 94)
EE4 C600-3
(11, 94)
EE4 C700-3
(11, 94)
EE4 C700-3
(12, 94)
EE4 C800-3
(10, 94)
EE4 C800-3
(9, 

In [10]:
BET_df

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,122.580645,134.921241,144.111021,149.905960,155.148598,159.042374,163.357996,167.257398,171.045656,174.283097,...,0.00,0.3904,599.0,0.66,0.200,0.460,278.0,0.009926,0.680672,2327
1,122.580645,134.921241,144.111021,149.905960,155.148598,159.042374,163.357996,167.257398,171.045656,174.283097,...,0.00,0.3904,599.0,0.66,0.200,0.460,278.0,0.019851,0.890756,2327
2,122.580645,134.921241,144.111021,149.905960,155.148598,159.042374,163.357996,167.257398,171.045656,174.283097,...,0.00,0.3904,599.0,0.66,0.200,0.460,278.0,0.029777,1.042017,2327
3,122.580645,134.921241,144.111021,149.905960,155.148598,159.042374,163.357996,167.257398,171.045656,174.283097,...,0.00,0.3904,599.0,0.66,0.200,0.460,278.0,0.064516,1.302521,2327
4,122.580645,134.921241,144.111021,149.905960,155.148598,159.042374,163.357996,167.257398,171.045656,174.283097,...,0.00,0.3904,599.0,0.66,0.200,0.460,278.0,0.121588,1.605042,2327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3780,334.112150,474.355404,533.067839,558.927932,583.336573,604.312661,625.288748,646.264835,660.650462,672.633302,...,0.07,0.3474,2390.0,1.21,0.726,0.484,273.0,0.369775,1.865878,2750
3781,334.112150,474.355404,533.067839,558.927932,583.336573,604.312661,625.288748,646.264835,660.650462,672.633302,...,0.07,0.3474,2390.0,1.21,0.726,0.484,273.0,0.540193,2.418731,2750
3782,334.112150,474.355404,533.067839,558.927932,583.336573,604.312661,625.288748,646.264835,660.650462,672.633302,...,0.07,0.3474,2390.0,1.21,0.726,0.484,273.0,0.697749,2.833370,2750
3783,334.112150,474.355404,533.067839,558.927932,583.336573,604.312661,625.288748,646.264835,660.650462,672.633302,...,0.07,0.3474,2390.0,1.21,0.726,0.484,273.0,0.874598,3.248010,2750


In [36]:
BET_df.to_csv('./E&E_Adding_bet_80_new.csv')

In [37]:
BET_df = pd.read_csv("E&E_Adding_bet_80_new.csv")
BET_df.dropna(axis=0,how="any",subset = ["BET","Vt","Vmic","BET_79"],inplace=True)
#BET_df.dropna(axis=0,how="any",subset = ["BET_34"],inplace=True)

In [25]:
BET_df.columns

Index(['Unnamed: 0', 'BET_0', 'BET_1', 'BET_2', 'BET_3', 'BET_4', 'BET_5',
       'BET_6', 'BET_7', 'BET_8', 'BET_9', 'BET_10', 'BET_11', 'BET_12',
       'BET_13', 'BET_14', 'BET_15', 'BET_16', 'BET_17', 'BET_18', 'BET_19',
       'BET_20', 'BET_21', 'BET_22', 'BET_23', 'BET_24', 'BET_25', 'BET_26',
       'BET_27', 'BET_28', 'BET_29', 'BET_30', 'BET_31', 'BET_32', 'BET_33',
       'BET_34', 'E', 'S', 'A', 'B', 'V', 'BET', 'Vt', 'Vmic', 'Vmeso',
       'Temp(K)', 'Pressure', 'Adsorp(mmol/g)', 'Index', 'Label'],
      dtype='object')

In [38]:
# data splitting
#from random import shuffle
from sklearn.model_selection import train_test_split
from numpy import random
def group_split(BET_df:pd.DataFrame,random_state = 42):
    indexs = list(set(BET_df["Index"].values.tolist()))
    #shuffle(indexs)
    _,test_index = train_test_split(indexs,test_size= 0.2,random_state =random_state)

    #test_index = random.choice(indexs,len(indexs)//5,replace=False).tolist()
    train_df = BET_df[~BET_df["Index"].isin(test_index)]
    test_df = BET_df[BET_df["Index"].isin(test_index)]
    return train_df,test_df

In [14]:
BET_df = BET_df[BET_df.notna()]

In [41]:
BET_df.describe

<bound method NDFrame.describe of       Unnamed: 0       BET_0       BET_1       BET_2       BET_3       BET_4  \
0              0  122.580645  131.262687  137.131441  142.831431  146.967106   
1              1  122.580645  131.262687  137.131441  142.831431  146.967106   
2              2  122.580645  131.262687  137.131441  142.831431  146.967106   
3              3  122.580645  131.262687  137.131441  142.831431  146.967106   
4              4  122.580645  131.262687  137.131441  142.831431  146.967106   
...          ...         ...         ...         ...         ...         ...   
3780        3780  334.112149  436.441778  498.367367  529.467066  545.506871   
3781        3781  334.112149  436.441778  498.367367  529.467066  545.506871   
3782        3782  334.112149  436.441778  498.367367  529.467066  545.506871   
3783        3783  334.112149  436.441778  498.367367  529.467066  545.506871   
3784        3784  334.112149  436.441778  498.367367  529.467066  545.506871   

     

In [40]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor,GradientBoostingRegressor,\
    BaggingRegressor,ExtraTreesRegressor,RandomForestRegressor
from lightgbm import LGBMRegressor  
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
  
n_estimators = [50,100,120,150,180,200,300]

# define different models#,
models = [
    #('SVR',SVR(max_iter=100000)),
    #('DT',DecisionTreeRegressor(random_state=42)),\
    # ('ADBR',AdaBoostRegressor(random_state=42)), 
    #("GBR",GradientBoostingRegressor(random_state=42)),\
    #('BG',BaggingRegressor(random_state=42,n_jobs=-1)),
    #('ETR',ExtraTreesRegressor(random_state=42,n_jobs=-1)),\
    #('RF',RandomForestRegressor(n_jobs=-1,random_state=42)),
    ('LGBM',LGBMRegressor(n_jobs = -1,random_state = 42)),\
    #('MLPR',MLPRegressor(random_state=42))
    #('BGLGBM',BaggingRegressor(LGBMRegressor(n_estimators = 200, n_jobs = -1,random_state = 42), random_state=42,n_jobs=-1)),\
    #('BGETR',BaggingRegressor(ExtraTreesRegressor(n_estimators = 180,random_state=42,n_jobs=6),random_state=42,n_jobs=-1))
    ]

# set search parameters grid for different models
para_grids = { 
    'SVR':{'kernel':['linear','poly','rbf','sigmoid','precomputed']},\
    'DT':{'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson']},\
    'ADBR':{'n_estimators':n_estimators,'learning_rate':[0.1,0.5,1,2],'loss':['linear','square','exponential']},\
    'GBR':{'n_estimators':n_estimators,'learning_rate':[0.1,0.5,1,2]},\
    'BG':{'n_estimators':[10,50,100]},\
    'ETR':{'n_estimators':n_estimators},\
    'RF':{'n_estimators':n_estimators},\
    'LGBM':{'num_leaves':[10,20,30,50,],'learning_rate': [0.05,0.1,0.5,1],'n_estimators':n_estimators},\
    'BGLGBM':{'n_estimators':[10,30,50]},\
    'BGETR':{'n_estimators':[10]},\
    'MLPR':{'hidden_layer_sizes':[(4,100),(5,100),(6,100)],
            "activation":["logistic","relu"],\
            "solver":['sgd', 'adam']}
    }

In [29]:
from sklearn.model_selection import GridSearchCV,cross_validate,GroupKFold
from sklearn.ensemble import ExtraTreesRegressor
from  sklearn.metrics import mean_squared_error,r2_score
from sklearn.utils import shuffle

def model_CV(train_x,train_y,groups,model,para_grid):

    out_cv = GroupKFold(n_splits = 5)
    result = GridSearchCV(model,para_grid,cv= out_cv.get_n_splits(groups =groups),
    scoring='neg_mean_squared_error', return_train_score=True,n_jobs=-1)
    result.fit(train_x,train_y)
    
    model_refit =model.set_params(**result.best_params_)
    train_cv = cross_validate(model_refit,train_x,train_y,groups = groups,cv =out_cv,scoring = ('r2', 'neg_mean_squared_error'))
    train_mse_cv = -train_cv['test_neg_mean_squared_error'].mean()
    train_r2_cv = train_cv['test_r2'].mean()
    
    return [train_r2_cv,train_mse_cv],result.best_params_

# model evaluation
def model_eval(model,test_x,test_y):
      
    test_pre = model.predict(test_x)
    test_r2 = r2_score(test_pre,test_y)
    test_mse = mean_squared_error(test_y,test_pre)
    return test_r2,test_mse

# comparing different models
def model_comparison(model_list,para_grids,feature_list,gas_list):
    gas_list = gas_list 
    input_feature = feature_list
    output = ['Adsorp(mmol/g)']
    result_total = []
    for gas in gas_list:
        if gas =='total':
            train_df_com = train_df
            test_df_com = test_df
            train_x = train_df_com[input_feature]
            test_x = test_df_com[input_feature]
            train_y = train_df_com[output].values
            test_y = test_df_com[output].values
            groups = train_df_com['Index'].values
            train_x, train_y, groups = shuffle(train_x, train_y, groups, random_state=42)
            
            for model_name, model in model_list:
                result, best_param = model_CV(train_x,train_y.squeeze(),groups,model,para_grids[model_name])
                model_refit = model.set_params(**best_param)
                model_refit.fit(train_x,train_y.squeeze())
                test_r2_total,test_mse_total = model_eval(model_refit,test_x,test_y.squeeze()) 
                for gases in gas_list[1:]:
                    test_df_com = test_df[test_df['Label']==gases]
                    test_xs = test_df_com[input_feature]
                    test_ys = test_df_com[output].values
                    test_r2,test_mse = model_eval(model_refit,test_xs,test_ys.squeeze()) 
                    result_total.append([gases,model_name+'_total',result[0],result[1],test_r2_total,test_mse_total,test_r2,test_mse,best_param])
                    print('Dataset {}, Algorithm {}, Test_r2 {}, Test_error {}'.format(gas,model_name+'_total',test_r2,test_mse))    
        else:
            train_df_com = train_df[train_df['Label']==gas]
            test_df_com = test_df[test_df['Label']==gas]
            train_x = train_df_com[input_feature]
            test_x = test_df_com[input_feature]
            train_y = train_df_com[output].values
            test_y = test_df_com[output].values
            groups = train_df_com['Index']
            train_x, train_y, groups = shuffle(train_x, train_y, groups, random_state=42)
            for model_name, model in model_list:
                result, best_param = model_CV(train_x,train_y.squeeze(),groups,model,para_grids[model_name])
                model_refit = model.set_params(**best_param)
                model_refit.fit(train_x,train_y.squeeze())
                test_r2,test_mse = model_eval(model_refit,test_x,test_y.squeeze()) 
                result_total.append([gas,model_name+'_separate',result[0],result[1],-1,-1, test_r2,test_mse,best_param])
                
                print('Dataset {}, Algorithm {}, Test_r2 {}, Test_error {}'.format(gas,model_name+'_total',test_r2,test_mse))     
    return result_total

In [42]:
import os
conditions = ['S','V','Temp(K)','Pressure']
base_feature = ["BET_"+str(i) for i in range(80)]+conditions
feature_list = [base_feature,base_feature+["BET"],base_feature+['Vt'],base_feature+["BET",'Vt'],\
    base_feature+["BET","Vt","Vmic","Vmeso"],["BET","Vt"]+conditions,["BET","Vt","Vmic","Vmeso"]+conditions]
columns = ['Gas','Model_name','CV_r2','CV_mse','test_r2_total_model','test_mse_by_total_model','test_r2_separa_model','test_mse_separa_model','best_param']
file_name = ['BET_Curve',"BET_Curve_BET","BET_Cureve_Vt","BET_Curve_BET_Vt","BET_Curve_BET_Vt_Vmic_Vmeso","BET_Vt","BET_Vt_Vmic_Vmeso"] #['BET_Curve','BET_Plus_Vt',"Vmic_Vmeso"]
gas_list = ['E&E']#,'CFCs','Methane','E&E'
  
for i in range(50): 
    train_df,test_df = group_split(BET_df,random_state=i)
    for j in range(len(feature_list)):
    
        results = model_comparison(models,para_grids, feature_list[j],gas_list)
        files_name = '80_point_curve_feature_'+file_name[j]+'_result_'+str(i)+'.csv'
        pd.DataFrame(results,columns = columns).to_csv(os.path.join('./8_using_BET_curve',files_name)) 
    print("The finish of {} th trial".format(i)) 

Dataset E&E, Algorithm LGBM_total, Test_r2 0.97260839156775, Test_error 0.17775495778473704
Dataset E&E, Algorithm LGBM_total, Test_r2 0.9742126362903083, Test_error 0.16683515628892107
Dataset E&E, Algorithm LGBM_total, Test_r2 0.9671757705347638, Test_error 0.2130604778364036
Dataset E&E, Algorithm LGBM_total, Test_r2 0.9713205424118346, Test_error 0.18773263987392824
Dataset E&E, Algorithm LGBM_total, Test_r2 0.9622120231885529, Test_error 0.24664956426100765
Dataset E&E, Algorithm LGBM_total, Test_r2 0.9516244662605279, Test_error 0.307307220614232
Dataset E&E, Algorithm LGBM_total, Test_r2 0.9594892764484956, Test_error 0.26929382314734174
The finish of 0 th trial
Dataset E&E, Algorithm LGBM_total, Test_r2 0.9488105080923873, Test_error 0.3592902970452879
Dataset E&E, Algorithm LGBM_total, Test_r2 0.9488580313661872, Test_error 0.36225445456857236
Dataset E&E, Algorithm LGBM_total, Test_r2 0.9472392930229372, Test_error 0.3546124085204691
Dataset E&E, Algorithm LGBM_total, Test_r2

# Post result treatments
 

In [32]:
import pandas as pd
file_name = ['BET_Curve',"BET_Curve_BET","BET_Cureve_Vt","BET_Curve_BET_Vt","BET_Curve_BET_Vt_Vmic_Vmeso","BET_Vt","BET_Vt_Vmic_Vmeso"] 

cal_columns= ["CV_r2","CV_mse","test_r2_separa_model","test_mse_separa_model"]
for j in range(len(file_name)):
    df_list = []
    for i in range(50):
        files_name = '80_point_curve_feature_'+file_name[j]+'_result_'+str(i)+'.csv'
        df_list.append(pd.read_csv(os.path.join('./8_using_BET_curve',files_name))[cal_columns] )
        pd.concat(df_list).groupby(level=0).mean().to_csv(os.path.join('./8_using_BET_curve',file_name[j]+'_mean.csv'))
        pd.concat(df_list).groupby(level=0).std().to_csv(os.path.join('./8_using_BET_curve',file_name[j]+'_std.csv'))

# 2_Predicting adsorption for each data point using the combination Vt, BET, Vmeso, and Vmic
the dataset for each separate gas will be smaller than previous one.

In [ ]:
import os
base_feature = ['V','S','L','BET',]
condition_feature = ['Temp(K)','Pressure']
combin_1 = ['Vt']
combin_2 = ["Vmeso"]
combin_3 = ['Vmic']
combin_4 = ['Vt',"Vmeso",]
combin_5 = ['Vt',"Vmic",]
combin_6 = ['Vt',"Vmic",'Vmeso',]
combin_7 = ["Vmic",'Vmeso',]

feature_list = [base_feature+condition_feature,base_feature+combin_1+condition_feature, \
    base_feature+combin_3+condition_feature, base_feature+combin_2+condition_feature,\
    base_feature+combin_4+condition_feature, base_feature+combin_5+condition_feature,\
    base_feature+combin_6+condition_feature, base_feature+combin_7+condition_feature, ]

columns = ['Gas','Model_name','CV_r2','CV_mse','test_r2_total_model','test_mse_by_total_model','test_r2_separa_model','test_mse_separa_model','best_param']


feature_set = ["BET","Vt","Vmeso","Vmic"]
gas_list = ['CO2','CFCs','Methane','E&E']
file_name = ['BET_only','BET_plut_Vt',"BET_Vmic","BET_Vmeso",'BET_Vt_Vmeso','BET_Vt_Vmic',"BET_Vt_Vmic_meso","BET_Vmic_meso"]


for i in range(10,15):
    train_df,test_df = pred_dataset(['CO2','Methane','Ethane&Ethylene','CFCs'],feature_set= feature_set)
    for j in range(len(feature_list)):
        results = model_comparison(models,para_grids, feature_list[j],gas_list)
        files_name = 'Four_gases_with_Vmic_'+file_name[j]+'_result_'+str(i)+'.csv'
        pd.DataFrame(results,columns = columns).to_csv(os.path.join('./2_Predicting_separate_gas_BET_Vt_Vmeso_Vmic',files_name))  
        #pd.DataFrame(results,columns = ['Gas','Algo','Train_erro','Test_error']).to_csv(os.path.join('./',files_name))   

# Post result treatment

In [24]:
import pandas as pd
file_name = ['BET_only','BET_plut_Vt',"BET_Vmic","BET_Vmeso",'BET_Vt_Vmeso','BET_Vt_Vmic',"BET_Vt_Vmic_meso","BET_Vmic_meso"]

cal_columns= ["CV_r2","CV_mse","test_r2_separa_model","test_mse_separa_model"]
for j in range(len(file_name)):
    df_list = []
    for i in range(11):
    
        files_name = 'Four_gases_with_Vmic_'+file_name[j]+'_result_'+str(i)+'.csv'
        df_list.append(pd.read_csv(os.path.join('./2_Predicting_separate_gas_BET_Vt_Vmeso_Vmic',files_name))[cal_columns] )
        pd.concat(df_list).groupby(level=0).mean().to_csv(os.path.join('./2_Predicting_separate_gas_BET_Vt_Vmeso_Vmic',file_name[j]+'_mean_new.csv'))
        pd.concat(df_list).groupby(level=0).std().to_csv(os.path.join('./2_Predicting_separate_gas_BET_Vt_Vmeso_Vmic',file_name[j]+'_std_new.csv'))


# Predicting the fitted parameters of adsorption isotherms

# Using only BET and Vt

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def pred_dataset(file_names, feature_set = feature_set):
    source_path = 'C:/Kai_Zhang/MachineLearning/Unified gas Adsorption/CO2_adsorption/new_data'
    train_df = pd.DataFrame()
    test_df = pd.DataFrame()
    for file_name in file_names:
        temp_data = pd.read_excel(os.path.join(source_path,file_name+'-fitting-02-02-2022.xlsx'),skiprows= 1 )
        temp_data = temp_data.dropna(axis=0,how = 'any',subset = feature_set)
        train_x,test_x = train_test_split(temp_data,test_size = 0.2)
        train_df = pd.concat([train_df,train_x],axis=0)
        test_df = pd.concat([test_df,test_x],axis =0)
    return train_df,test_df

In [ ]:
from sklearn.model_selection import GridSearchCV,cross_validate,GroupKFold
from sklearn.ensemble import ExtraTreesRegressor
from  sklearn.metrics import mean_squared_error,r2_score
from sklearn.utils import shuffle

def model_CV(train_x,train_y,model,para_grid):

    
    result = GridSearchCV(model,para_grid,cv= 5,
    scoring='neg_mean_squared_error', return_train_score=True,n_jobs=-1)
    result.fit(train_x,train_y)
    
    model_refit =model.set_params(**result.best_params_)
    train_cv = cross_validate(model_refit,train_x,train_y,cv =5,scoring = ('r2', 'neg_mean_squared_error'))
    train_mse_cv = -train_cv['test_neg_mean_squared_error'].mean()
    train_r2_cv = train_cv['test_r2'].mean()
    
    return [train_r2_cv,train_mse_cv],result.best_params_

# model evaluation
def model_eval(model,test_x,test_y):
      
    test_pre = model.predict(test_x)
    test_r2 = r2_score(test_pre,test_y)
    test_mse = mean_squared_error(test_y,test_pre)
    return test_r2,test_mse

# comparing different models
def model_comparison(model_list,para_grids,feature_list,gas_list):
    gas_list = gas_list 
    input_feature = feature_list
    output = ['Adsorp(mmol/g)']
    result_total = []

    for gas in gas_list:
        
        if gas =='total':

            train_df_com = train_df
            test_df_com = test_df
            train_x = train_df_com[input_feature]
            test_x = test_df_com[input_feature]
            train_y = train_df_com[output].values
            test_y = test_df_com[output].values
            
            train_x, train_y = shuffle(train_x, train_y,random_state=42)
            
            for model_name, model in model_list:

                
                result, best_param = model_CV(train_x,train_y.squeeze(),model,para_grids[model_name])
                model_refit = model.set_params(**best_param)
                model_refit.fit(train_x,train_y.squeeze())
                test_r2_total,test_mse_total = model_eval(model_refit,test_x,test_y.squeeze()) 
                for gases in gas_list[1:]:
                    test_df_com = test_df[test_df['Label']==gases]
                    test_xs = test_df_com[input_feature]
                    test_ys = test_df_com[output].values
                    test_r2,test_mse = model_eval(model_refit,test_xs,test_ys.squeeze()) 
                    result_total.append([gases,model_name+'_total',result[0],result[1],test_r2_total,test_mse_total,test_r2,test_mse,best_param])

                    print('Dataset {}, Algorithm {}, Test_r2 {}, Test_error {}'.format(gas,model_name+'_total',test_r2,test_mse))

            
        else:
            
            train_df_com = train_df[train_df['Label']==gas]
            test_df_com = test_df[test_df['Label']==gas]
            train_x = train_df_com[input_feature]
            test_x = test_df_com[input_feature]
            train_y = train_df_com[output].values
            test_y = test_df_com[output].values
            
            train_x, train_y = shuffle(train_x, train_y, random_state=42)
           
            for model_name, model in model_list:

                result, best_param = model_CV(train_x,train_y.squeeze(),model,para_grids[model_name])
                model_refit = model.set_params(**best_param)
                model_refit.fit(train_x,train_y.squeeze())
                test_r2,test_mse = model_eval(model_refit,test_x,test_y.squeeze()) 
                result_total.append([gas,model_name+'_separate',result[0],result[1],-1,-1, test_r2,test_mse,best_param])
                
                print(print('Dataset {}, Algorithm {}, Test_r2 {}, Test_error {}'.format(gas,model_name+'_total',test_r2,test_mse))
)
                
    return result_total